# agora_api

> SDK to interact with Agora private api

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastcore.utils import *
import os
import httpx
from typing import Dict, Any, List
from time import time, sleep
import json
from httpx import HTTPError


In [ ]:
#| export

base_url = 'https://zues.searchagora.com/api/v1'

In [ ]:
#| export

#| export
class Agora:
    "Client for interacting with the Agora API"
    def __init__(self,
                 api_key: str = None, # The API key for the Agora account
                 base_url: str = "https://zues.searchagora.com/api/v1"): # The Agora API base URL
        self.api_key = api_key or os.environ.get("AGORA_API_KEY")
        # TODO: maybe change this for a decorator so search_trial can be used without api_key
        # if not self.api_key:
        #     raise ValueError("The api_key client option must be set either by passing api_key to the client or by setting the AGORA_API_KEY environment variable")
        self.base_url = base_url
        self._httpx_client = httpx.Client()
        self._httpx_client.headers.update({"Authorization": f"Bearer {self.api_key}"})



In [ ]:
a = Agora()

In [ ]:
#| export

@patch
def search_trial(self: Agora,
                 query: str, # Search query text
                price_min: int = 0, # Minimum price for filtering products
                price_max: int = None, # Maximum price for filtering products
                sort: str = None, # Sorting field: price:relevance
                order: str = None): # Sorting order: asc or desc
    """
    Search for products using the trial endpoint.
    
    Args:
        query (str): Search query text
        price_min (int, optional): Minimum price for filtering products (default: 0)
        price_max (int, optional): Maximum price for filtering products
        sort (str, optional): Sorting field: price:relevance
        order (str, optional): Sorting order: asc or desc
        
    Returns:
        dict: Search results with products matching the query
    
    Example:
        search_trial("shoes", [100, 1000], "price:relevance", "desc")
    """
    endpoint = f'{self.base_url}/search/trial'
    
    # Build query parameters
    params = {'q': query}

    # Handle price range parameters
    if price_min != 0 and price_max is not None:
        params['priceRange'] = [price_min, price_max]
        
    if sort: params['sort'] = sort
    if order: params['order'] = order
    
    # Make the request
    return httpx.get(endpoint, params=params)

In [ ]:

r = a.search_trial(sort= 'price:relevance', order='asc',query='glasses', price_max=1000)
p = r.json()['Products'][0]
p

{'name': 'Kaleidoscope Glasses',
 'storeName': 'Costumes, Etc...',
 'brand': 'Western Fashion',
 '_id': '677df599770698bbe867b39f',
 'slug': 'copy-of-kaleidscope-goggles-6b527f77-bd25-4846-ad2b-344fd2cc95ef-1736308121269',
 'price': 15,
 'isVerified': False,
 'isBoosted': False,
 'source': 'shopify',
 'images': ['https://cdn.shopify.com/s/files/1/0292/5599/1401/products/69132-4-1_1.jpg?v=1625761508'],
 'url': 'https://costumesetc.com/products/copy-of-kaleidscope-goggles',
 'agoraScore': 81,
 'priceHistory': [{'price': 15,
   'date': '2025-01-08T03:48:41.347Z',
   '_id': '677df5a3770698bbe867bcdc'}],
 'averageRating': 4.5,
 'discountVal': 0}

In [ ]:
json.dumps(p)

'{"name": "Headphones", "storeName": "The Baby Cubby", "brand": "Tonies", "_id": "6674b4aab49534201189688d", "slug": "tonies-headphones-08c2c9a1-b3af-4883-aebf-2a283a1fc0fd-1718924458056", "price": 29.99, "isVerified": false, "isBoosted": false, "source": "shopify", "images": ["https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-PDP-Assets-Gray-1.jpg?v=1637054130", "https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-PDP-Assets-Gray-2.jpg?v=1637054132", "https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-PDP-Assets-Gray-3.jpg?v=1637054134", "https://cdn.shopify.com/s/files/1/0467/0649/1556/products/blue_8b41cdb7-1f12-4b3d-b43c-8aac607377f3.jpg?v=1637336366", "https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-PDP-Assets-Pink-1.jpg?v=1637336366", "https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Reddd.jpg?v=1637336366"], "url": "https://babycubby.com/products/tonies-headphones", "agoraScore": 92, "priceHistory": 

## Methods

In [ ]:
#| export

@patch
def _request(self: Agora, 
             method: str, # The HTTP method to use
             path: str, # The path to request
             timeout: int = 10, # Timeout for the request in s
             **kwargs) -> Dict[str, Any]:
    "Makes an authenticated request to Agora API"
    url = f"{self.base_url}/{path}"
    return  self._httpx_client.request(method, url, timeout=timeout, **kwargs)


@patch
def text_search(self: Agora, 
                query: str, # Search query text
                count: int = 20, # Number of products per page (default: 20, max: 250)
                page: int = 1, # Page number for pagination (default: 1)
                price_min: int = 0, # Minimum price for filtering products
                price_max: int = None, # Maximum price for filtering products
                sort: str = None, # Sorting field: price:relevance
                order: str = None, # Sorting order: asc or desc
                image_id: str = None): # Image search identifier
    """
    Search for products with full functionality.
    
    Args:
        query (str): Search query text
        count (int, optional): Number of products per page (default: 20, max: 250)
        page (int, optional): Page number for pagination (default: 1)
        price_min (int, optional): Minimum price for filtering products (default: 0)
        price_max (int, optional): Maximum price for filtering products
        sort (str, optional): Sorting field: price:relevance
        order (str, optional): Sorting order: asc or desc
        image_id (str, optional): Image search identifier
        
    Returns:
        dict: Search results with products matching the query
    
    Example:
        text_search("red shoes", count=10, page=1, price_min=50, price_max=200, sort="price", order="asc")
    """
    
    # Build query parameters
    params = {'q': query, 'count': count, 'page': page}
    
    # Handle price range parameters
    if price_min != 0 and price_max is not None:
        params['priceRange'] = [price_min, price_max]
        
    if sort: params['sort'] = sort
    if order: params['order'] = order
    if image_id: params['imageId'] = image_id
    
    return self._request('GET', path='search', params=params)
    

In [ ]:
#| export

@patch
def get_product_detail(self: Agora, 
                      slug: str): # The unique identifier of the product to retrieve
    """
    Retrieve detailed information about a specific product.
    
    Args:
        slug (str): The unique identifier of the product to retrieve
        
    Returns:
        dict: Detailed information about the requested product
    
    Example:
        agora.get_product_detail("calzuro-without-pistachio-eb12f468-48a2-48af-9f5e-3fda5f6c135c-1708446961787")
    """
    # Build query parameters
    params = {'slug': slug}
    
    # Make the request
    return self._request('GET', path='product-detail', params=params)


In [ ]:

# get_product_detail(token, p.slug)

In [ ]:
#| export

@patch
def create_cart(self: Agora, 
                custom_user_id: str = None, # Unique identifier for the user
                items: List[Dict] = None): # List of items to add to the cart
    """
    Create a new cart for a user.
    
    Args:
        custom_user_id (str, optional): Unique identifier for the user
        items (list, optional): List of items to add to the cart
            Each item should be a dict with:
            - variantId (int): Variant ID of the product
            - product (int/str): Product ID
            - quantity (int): Quantity of the product
            
    Returns:
        dict: Response with cart creation status
    
    Example:
        agora.create_cart("user123", [
            {"variantId": 123, "product": "678f71a9356a36f784ee2e88", "quantity": 1}
        ])
    """
    # Add custom user ID if provided
    headers = {}
    if custom_user_id:
        headers['customuserid'] = custom_user_id
    
    # Prepare request body
    data = {}
    if items:
        data['items'] = items
    
    # Make the request
    return self._request('POST', path='cart', headers=headers, json=data)

In [ ]:
#| export

@patch
def add_to_cart(self: Agora, 
                product_id: str, # ID of the product to add
                variant_id: str, # Variant ID of the product
                quantity: int = 1, # Quantity of the product (default: 1)
                custom_user_id: str = None): # Unique identifier for the user
    """
    Add a product to an existing cart.
    
    Args:
        product_id (str/int): ID of the product to add
        variant_id (str/int): Variant ID of the product
        quantity (int, optional): Quantity of the product (default: 1)
        custom_user_id (str, optional): Unique identifier for the user
        
    Returns:
        dict: Response with cart update status
    
    Example:
        agora.add_to_cart("678f71a9356a36f784ee2e88", "2061038485517", 2, "user123")
    """
    # Add custom user ID if provided
    headers = {}
    if custom_user_id:
        headers['customuserid'] = custom_user_id
    
    # Prepare request body
    data = {
        "product": {
            "product": product_id,
            "variantId": variant_id,
            "quantity": quantity
        }
    }
    
    # Make the PUT request
    return self._request('PUT', path='cart', headers=headers, json=data)

In [ ]:
#| export

@patch
def create_order(self: Agora, 
                encrypted_payment_info: str, # Encrypted payment information
                shipping_address: Dict[str, str], # Dictionary containing shipping address details
                current_user: Dict[str, str]): # Dictionary containing user information
    """
    Create a new order from cart items.
    
    Args:
        encrypted_payment_info (str): Encrypted payment information
        shipping_address (dict): Dictionary containing shipping address details:
            - addressFirst (str): Street address
            - city (str): City name
            - state (str): State/province
            - country (str): Country name
            - addressName (str): Name associated with address
            - zipCode (str): Postal/ZIP code
        current_user (dict): Dictionary containing user information:
            - firstname (str): User's first name
            - lastname (str): User's last name
            - email (str): User's email address
            - _id (str): User's ID
            
    Returns:
        dict: Order creation response with success status and order ID
    
    Example:
        agora.create_order(
            "encrypted_data",
            {
                "addressFirst": "123 Main St",
                "city": "New York",
                "state": "NY",
                "country": "US",
                "addressName": "Home",
                "zipCode": "10001"
            },
            {
                "firstname": "John",
                "lastname": "Doe",
                "email": "john@example.com",
                "_id": "user123"
            }
        )
    """
    # Prepare request body
    data = {
        "encryptedPaymentInfo": encrypted_payment_info,
        "shippingAddress": shipping_address,
        "currentUser": current_user
    }
    
    # Make the POST request
    return self._request('POST', path='order', json=data)

In [ ]:
#| export

@patch
def track_order(self: Agora, 
               order_id: str): # Unique identifier of the order to track
    """
    Track an existing order by its ID.
    
    Args:
        order_id (str): Unique identifier of the order to track
        
    Returns:
        dict: Detailed order information including items, status, and shipping details
    
    Example:
        agora.track_order("67c8577b3e370f07d12c7722")
    """
    # Make the GET request
    return self._request('GET', path=f'order-tracking/{order_id}')

In [ ]:
#| export 

@patch
def refresh_token(self: Agora, 
                 refresh_token_str: str): # The refresh token to validate
    """
    Refresh API key and token by providing a valid refresh token.
    
    Args:
        refresh_token_str (str): The refresh token to validate and retrieve
                               a new API key and refresh token
        
    Returns:
        dict: Object containing new API key, refresh token, and expiration time
              with the following structure:
              {
                "status": "success",
                "data": {
                  "apiKey": "...",
                  "refreshToken": "...",
                  "expiresAt": "..."
                }
              }
    
    Raises:
        ValueError: If the refresh token is invalid or missing
    """
    # Prepare request body
    data = {"refreshToken": refresh_token_str}
    
    # Make the POST request (using direct httpx call since we don't need auth here)
    r = httpx.post(f'{self.base_url}/refresh-token', json=data)
    response = dict2obj(r.json())
    
    # Check for errors
    if r.status_code != 200 or response.status == "error":
        raise ValueError(response.message)
        
    return response

In [ ]:
#| export

@patch
def as_tools(self:Agora):
    "Return list of available tools for AI agents"
    return [
        self.search_trial,
        self.get_product_detail,
        self.create_cart,
        self.add_to_cart,
        self.create_order,
        self.track_order,
        self.refresh_token,
    ]

## AI Agent

In [ ]:
from claudette import *
os.environ['ANTHROPIC_LOG'] = 'debug'


In [ ]:
chat = Chat(models[1], sp='You are a helpful assistant that can search for products in a store',
             tools=[search_trial])
chat.toolloop('can you search for headp`hones in the price range of 100 to 200', trace_func=print)
chat.h

[{'role': 'user', 'content': 'can you search for headp`hones in the price range of 100 to 200'}, {'role': 'assistant', 'content': [TextBlock(citations=None, text="I'll search for headphones in the price range of $100 to $200 for you.", type='text'), ToolUseBlock(id='toolu_01Wf3Ms8DYoMQvXjAxj5KRgX', input={'query': 'headphones', 'price_range': [100, 200]}, name='search_trial', type='tool_use')]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_01Wf3Ms8DYoMQvXjAxj5KRgX', 'content': "{'status': 'success', 'Products': [{'name': 'Headphones', 'storeName': 'The Baby Cubby', 'brand': 'Tonies', '_id': '6674b4aab49534201189688d', 'slug': 'tonies-headphones-08c2c9a1-b3af-4883-aebf-2a283a1fc0fd-1718924458056', 'price': 29.99, 'isVerified': False, 'isBoosted': False, 'source': 'shopify', 'images': ['https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-PDP-Assets-Gray-1.jpg?v=1637054130', 'https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-

[{'role': 'user',
  'content': 'can you search for headp`hones in the price range of 100 to 200'},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text="I'll search for headphones in the price range of $100 to $200 for you.", type='text'),
   ToolUseBlock(id='toolu_01Wf3Ms8DYoMQvXjAxj5KRgX', input={'query': 'headphones', 'price_range': [100, 200]}, name='search_trial', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01Wf3Ms8DYoMQvXjAxj5KRgX',
    'content': "{'status': 'success', 'Products': [{'name': 'Headphones', 'storeName': 'The Baby Cubby', 'brand': 'Tonies', '_id': '6674b4aab49534201189688d', 'slug': 'tonies-headphones-08c2c9a1-b3af-4883-aebf-2a283a1fc0fd-1718924458056', 'price': 29.99, 'isVerified': False, 'isBoosted': False, 'source': 'shopify', 'images': ['https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-PDP-Assets-Gray-1.jpg?v=1637054130', 'https://cdn.shopify.com/s/files/1/0467/0649/1556/

In [ ]:
def render_product(item: str): # item is a json string
    """
    Renders a product card UI component displaying product details clearly and attractively.
    Expects `item` as a JSON-formatted string with these keys:
    - `name`: Product name (str)
    - `price`: Product price (float or str)
    - `brand`: Brand name (str)
    - `storeName`: Store name (str)
    - `agoraScore`: Product rating score out of 100 (int or float)
    - `images`: List of image URLs (list[str]), first image used as main display
    - `_id`: Unique identifier for the product (str)
    """
    print('Render product !!!!')

## Sync chat with tools

In [ ]:
chat = Chat(models[1], sp='You are a helpful assistant that can search for products in a store', 
            tools=[search_trial, render_product]) 
r = chat('Can you search for headphones in the price range of 100 to 200')
r, chat.h

(Message(id='msg_01Wvh5gwAQFnVCfL5NFeWi4C', content=[TextBlock(citations=None, text="I'll search for headphones in the price range of $100 to $200 for you.", type='text'), ToolUseBlock(id='toolu_01W8ZPtVFAw8pST7PFMew7Gm', input={'query': 'headphones', 'price_range': [100, 200]}, name='search_trial', type='tool_use')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=In: 847; Out: 100; Cache create: 0; Cache read: 0; Total: 947),
 [{'role': 'user',
   'content': 'Can you search for headphones in the price range of 100 to 200'},
  {'role': 'assistant',
   'content': [TextBlock(citations=None, text="I'll search for headphones in the price range of $100 to $200 for you.", type='text'),
    ToolUseBlock(id='toolu_01W8ZPtVFAw8pST7PFMew7Gm', input={'query': 'headphones', 'price_range': [100, 200]}, name='search_trial', type='tool_use')]},
  {'role': 'user',
   'content': [{'type': 'tool_result',
     'tool_use_id': 'toolu_

In [ ]:
r = chat()
r, chat.h

Render product !!!!


(Message(id='msg_01FFCr92UrQch5JczKcyXJ2v', content=[TextBlock(citations=None, text='Based on your search for headphones in the $100-$200 price range, I found one product that matches your criteria:', type='text'), ToolUseBlock(id='toolu_01GLFH2wNfbSR9tvoNZypQCg', input={'item': '{"name": "Sonic Headphones (Black)", "storeName": "TC Running Co", "brand": "Suunto", "_id": "677e58f7770698bbe86cf7d7", "price": 150, "images": ["https://cdn.shopify.com/s/files/1/1804/7057/files/suunto-sonic-black-side-3-1280x1280px.webp?v=1717101867"], "agoraScore": 92}'}, name='render_product', type='tool_use')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=In: 5584; Out: 198; Cache create: 0; Cache read: 0; Total: 5782),
 [{'role': 'user',
   'content': 'Can you search for headphones in the price range of 100 to 200'},
  {'role': 'assistant',
   'content': [TextBlock(citations=None, text="I'll search for headphones in the price ran

In [ ]:
r = chat()
r, chat.h

(Message(id='msg_01F29q9U8h9qZdZxYC2MHowE', content=[TextBlock(citations=None, text='I found one product that matches your price range of $100-$200:\n\n**Sonic Headphones (Black)**\n- Brand: Suunto\n- Price: $150.00\n- Store: TC Running Co\n- Rating: 92/100\n\nThese are the only headphones currently available in your specified price range. Would you like me to search for headphones in a different price range, or would you like more information about this particular model?', type='text')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=In: 5794; Out: 101; Cache create: 0; Cache read: 0; Total: 5895),
 [{'role': 'user',
   'content': 'Can you search for headphones in the price range of 100 to 200'},
  {'role': 'assistant',
   'content': [TextBlock(citations=None, text="I'll search for headphones in the price range of $100 to $200 for you.", type='text'),
    ToolUseBlock(id='toolu_01W8ZPtVFAw8pST7PFMew7Gm', input={'

## Async chat with tools

In [ ]:
chat = AsyncClient(models[1])
await chat('Can you search for headphones in the price range of 100 to 200', sp='You are a helpful assistant that can search for products in a store',tools=[search_trial, render_product])

AttributeError: 'AsyncClient' object has no attribute 'toolloop'

In [ ]:
import json
from fasthtml.common import *
from monsterui.all import *

In [ ]:
from fasthtml.jupyter import *

In [ ]:
def ProductCard(
    item: str # JSON string containing product details (`name`, `price`, `brand`, `storeName`, `agoraScore`, `images`, `_id`)
):
    """
    Renders a product card UI component displaying product details clearly and attractively.
    Expects `item` as a JSON-formatted string with these keys:
    - `name`: Product name (str)
    - `price`: Product price (float or str)
    - `brand`: Brand name (str)
    - `storeName`: Store name (str)
    - `agoraScore`: Product rating score out of 100 (int or float)
    - `images`: List of image URLs (list[str]), first image used as main display
    - `_id`: Unique identifier for the product (str)
    """
    item = dict2obj(json.loads(item))
    return to_xml(
        Card(
            DivHStacked(
                # Left column with image
                Div(
                    Img(src=item.images[0], alt=item.name, cls="max-h-52 object-contain")
                ),
                # Right column with product details
                DivVStacked(
                    H3(item.name),
                    Strong(f"${item.price}"),
                    Div(f"Brand: {item.brand}"),
                    Div(f"Store: {item.storeName}"),
                    Div(f"Rating: {item.agoraScore}/100"),
                    Button("View Details", id=f"view-{item._id}")
                )
            )
        )
    )

In [ ]:
from functools import partial
app, rt = fast_app(hdrs=Theme.blue.headers())
Show = partial(HTMX, app=app, link=True)
server = JupyUvi(app, port=8001)
data = search_trial('headphones', price_range=[100, 200])
item = data['Products'][0]
ProductCard(json.dumps(item))

```html
<div class="uk-card ">
  <div class="uk-card-body space-y-6">
    <div class="flex flex-row items-center space-x-4">
      <div>
<img src="https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-PDP-Assets-Gray-1.jpg?v=1637054130" alt="Headphones" class="max-h-52 object-contain">      </div>
      <div class="flex flex-col items-center space-y-4">
        <h3 class="uk-h3 ">Headphones</h3>
<strong class="font-bold ">$29.99</strong>        <div>Brand: Tonies</div>
        <div>Store: The Baby Cubby</div>
        <div>Rating: 92/100</div>
<button type="submit" id="view-6674b4aab49534201189688d" class="uk-btn uk-btn-default" name="view-6674b4aab49534201189688d">View Details</button>      </div>
    </div>
  </div>
</div>

```

In [ ]:
Show(ProductCard(json.dumps(item)))

In [ ]:
chat = Chat(models[1], sp='''You are a helpful assistant that can search for products in a store. 
            Whenever you need to display a product, use the ProductCard tool.''',
             tools=[search_trial, ProductCard])
chat.toolloop('can you search for headphones in the price range of 100 to 200', trace_func=print)

[{'role': 'user', 'content': 'can you search for headphones in the price range of 100 to 200'}, {'role': 'assistant', 'content': [TextBlock(citations=None, text="I'll search for headphones in the price range of $100 to $200 for you.", type='text'), ToolUseBlock(id='toolu_01PyPi77BGYsWt8B7ugdpNUN', input={'query': 'headphones', 'price_range': [100, 200]}, name='search_trial', type='tool_use')]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_01PyPi77BGYsWt8B7ugdpNUN', 'content': "{'status': 'success', 'Products': [{'name': 'Headphones', 'storeName': 'The Baby Cubby', 'brand': 'Tonies', '_id': '6674b4aab49534201189688d', 'slug': 'tonies-headphones-08c2c9a1-b3af-4883-aebf-2a283a1fc0fd-1718924458056', 'price': 29.99, 'isVerified': False, 'isBoosted': False, 'source': 'shopify', 'images': ['https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-PDP-Assets-Gray-1.jpg?v=1637054130', 'https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-P

I only found one headphone option within your specified price range of &#36;100-&#36;200:

- Sonic Headphones (Black) by Suunto
- Price: &#36;150
- Store: TC Running Co
- Rating: 92/100

Would you like me to search for more headphone options with different price ranges, or would you like more information about this particular model?

<details>

- id: `msg_01G79UST3iu3kJj4h91ankYk`
- content: `[{'citations': None, 'text': 'I only found one headphone option within your specified price range of $100-$200:\n\n- Sonic Headphones (Black) by Suunto\n- Price: $150\n- Store: TC Running Co\n- Rating: 92/100\n\nWould you like me to search for more headphone options with different price ranges, or would you like more information about this particular model?', 'type': 'text'}]`
- model: `claude-3-7-sonnet-20250219`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 6167, 'output_tokens': 86}`

</details>

In [ ]:
chat.h

[{'role': 'user',
  'content': 'can you search for headphones in the price range of 100 to 200'},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text="I'll search for headphones in the price range of $100 to $200 for you.", type='text'),
   ToolUseBlock(id='toolu_01PyPi77BGYsWt8B7ugdpNUN', input={'query': 'headphones', 'price_range': [100, 200]}, name='search_trial', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01PyPi77BGYsWt8B7ugdpNUN',
    'content': "{'status': 'success', 'Products': [{'name': 'Headphones', 'storeName': 'The Baby Cubby', 'brand': 'Tonies', '_id': '6674b4aab49534201189688d', 'slug': 'tonies-headphones-08c2c9a1-b3af-4883-aebf-2a283a1fc0fd-1718924458056', 'price': 29.99, 'isVerified': False, 'isBoosted': False, 'source': 'shopify', 'images': ['https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-PDP-Assets-Gray-1.jpg?v=1637054130', 'https://cdn.shopify.com/s/files/1/0467/0649/1556/p

## Our own tool loop

In [ ]:
from claudette.core import Chat, mk_toolres, mk_msg, mk_msgs, contents, call_func, mk_ns, find_block, ToolUseBlock

# Define the model and system prompt
model = 'claude-3-7-sonnet-20250219'
sp = "You are a product search assistant."

# Assume search_products and render_product are already defined in our environment.
# Initialize the Chat interface with our tools and a default continuation prompt.
chat = Chat(model, sp=sp, tools=[search_trial, render_product], cont_pr="continue")

# Reset the conversation history.
chat.h = []
chat  # This will be printed automatically.

No results yet

In [ ]:
# Send the initial user prompt that should trigger a tool call.
r = chat("Show me running shoes by Nike")
r  # This will display the assistant's response, likely containing a tool request.

I'll help you find Nike running shoes. Let me search for those for you right away.

<details>

- id: `msg_01UfwUNEQfUyLukaArEeE3Mx`
- content: `[{'citations': None, 'text': "I'll help you find Nike running shoes. Let me search for those for you right away.", 'type': 'text'}, {'id': 'toolu_01GA4MQ1XrR4C6pQRcxx9Auk', 'input': {'query': 'Nike running shoes'}, 'name': 'search_trial', 'type': 'tool_use'}]`
- model: `claude-3-7-sonnet-20250219`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 830, 'output_tokens': 75}`

</details>

In [ ]:
chat.h

[{'role': 'user', 'content': 'Show me running shoes by Nike'},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text="I'll help you find Nike running shoes. Let me search for those for you right away.", type='text'),
   ToolUseBlock(id='toolu_01GA4MQ1XrR4C6pQRcxx9Auk', input={'query': 'Nike running shoes'}, name='search_trial', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01GA4MQ1XrR4C6pQRcxx9Auk',
    'content': "{'status': 'success', 'Products': [{'name': 'Nike Running Shoes Sneakers CW3398-102 Womens Size 11 White Air Max Bella TR 4', 'storeName': 'Goodfair', 'brand': 'Nike', '_id': '6745d4c5ecd6d824cc57c98a', 'slug': 'nike-running-shoes-sneakers-cw3398-102-womens-size-11-white-air-max-bella-tr-4-29cd1d62-c9cf-4e08-91f6-0bd041500200-1732629701027', 'price': 41.99, 'isVerified': False, 'isBoosted': False, 'source': 'shopify', 'images': ['https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_8f6713b3-9859-4a

In [ ]:
r

I'll help you find Nike running shoes. Let me search for those for you right away.

<details>

- id: `msg_01UfwUNEQfUyLukaArEeE3Mx`
- content: `[{'citations': None, 'text': "I'll help you find Nike running shoes. Let me search for those for you right away.", 'type': 'text'}, {'id': 'toolu_01GA4MQ1XrR4C6pQRcxx9Auk', 'input': {'query': 'Nike running shoes'}, 'name': 'search_trial', 'type': 'tool_use'}]`
- model: `claude-3-7-sonnet-20250219`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 830, 'output_tokens': 75}`

</details>

In [ ]:
# Look for a ToolUseBlock in the response.
tool_req = None
for block in r.content:
    if isinstance(block, ToolUseBlock):
        tool_req = block
        break
tool_req  # Should display the tool request details.

ToolUseBlock(id='toolu_01GA4MQ1XrR4C6pQRcxx9Auk', input={'query': 'Nike running shoes'}, name='search_trial', type='tool_use')

In [ ]:
tc = first([o for o in r.content if isinstance(o, ToolUseBlock)])
tc.name, tc.input, tc.id

('search_trial',
 {'query': 'Nike running shoes'},
 'toolu_01GA4MQ1XrR4C6pQRcxx9Auk')

In [ ]:
tr = first(chat.h[-1].content)
tr.type, tr.tool_use_id, tr.content

('tool_result',
 'toolu_01GA4MQ1XrR4C6pQRcxx9Auk',
 "{'status': 'success', 'Products': [{'name': 'Nike Running Shoes Sneakers CW3398-102 Womens Size 11 White Air Max Bella TR 4', 'storeName': 'Goodfair', 'brand': 'Nike', '_id': '6745d4c5ecd6d824cc57c98a', 'slug': 'nike-running-shoes-sneakers-cw3398-102-womens-size-11-white-air-max-bella-tr-4-29cd1d62-c9cf-4e08-91f6-0bd041500200-1732629701027', 'price': 41.99, 'isVerified': False, 'isBoosted': False, 'source': 'shopify', 'images': ['https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_8f6713b3-9859-4aff-9118-50541af25f47.jpg?v=1693248497', 'https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_3c1bbfe7-1fe9-4df1-8d55-1ddc48d3bd8d.jpg?v=1693248497', 'https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_ae1975b0-2b04-459e-8443-c178663631d7.jpg?v=1693248497', 'https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_846093bc-814d-4a4f-869f-da942285943d.jpg?v=1693248497', 'https://cdn.shopify.com/s/files/1/0077/17

In [ ]:
tr =  [tr for tr in chat.h[-1].content if tr.type == 'tool_result' and tr.tool_use_id == tc.id]
assert len(tr) == 1

In [ ]:
first(tr).content

"{'status': 'success', 'Products': [{'name': 'Nike Running Shoes Sneakers CW3398-102 Womens Size 11 White Air Max Bella TR 4', 'storeName': 'Goodfair', 'brand': 'Nike', '_id': '6745d4c5ecd6d824cc57c98a', 'slug': 'nike-running-shoes-sneakers-cw3398-102-womens-size-11-white-air-max-bella-tr-4-29cd1d62-c9cf-4e08-91f6-0bd041500200-1732629701027', 'price': 41.99, 'isVerified': False, 'isBoosted': False, 'source': 'shopify', 'images': ['https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_8f6713b3-9859-4aff-9118-50541af25f47.jpg?v=1693248497', 'https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_3c1bbfe7-1fe9-4df1-8d55-1ddc48d3bd8d.jpg?v=1693248497', 'https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_ae1975b0-2b04-459e-8443-c178663631d7.jpg?v=1693248497', 'https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_846093bc-814d-4a4f-869f-da942285943d.jpg?v=1693248497', 'https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_7bec3532-a075-458e-bdf3-995fbc0e

In [ ]:
tc = first([o for o in chat.h[-1].content if isinstance(o, ToolResult)])


NameError: name 'ToolResult' is not defined

In [ ]:
tc = tool_calls[0]
tc.name, tc.input

('search_trial', {'query': 'Nike running shoes'})

[{'role': 'user', 'content': 'Show me running shoes by Nike'},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text="I'll help you find Nike running shoes. Let me search for those for you right away.", type='text'),
   ToolUseBlock(id='toolu_01GA4MQ1XrR4C6pQRcxx9Auk', input={'query': 'Nike running shoes'}, name='search_trial', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01GA4MQ1XrR4C6pQRcxx9Auk',
    'content': "{'status': 'success', 'Products': [{'name': 'Nike Running Shoes Sneakers CW3398-102 Womens Size 11 White Air Max Bella TR 4', 'storeName': 'Goodfair', 'brand': 'Nike', '_id': '6745d4c5ecd6d824cc57c98a', 'slug': 'nike-running-shoes-sneakers-cw3398-102-womens-size-11-white-air-max-bella-tr-4-29cd1d62-c9cf-4e08-91f6-0bd041500200-1732629701027', 'price': 41.99, 'isVerified': False, 'isBoosted': False, 'source': 'shopify', 'images': ['https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_8f6713b3-9859-4a

In [ ]:
# Build a namespace including our tools.
ns = mk_ns(search_trial, render_product)
# Use call_func to execute the tool function requested.
tool_result = call_func(tool_req.name, tool_req.input, ns)
tool_result  # This should print the result from search_products.

{'status': 'success',
 'Products': [{'name': 'Nike Running Shoes Sneakers CW3398-102 Womens Size 11 White Air Max Bella TR 4',
   'storeName': 'Goodfair',
   'brand': 'Nike',
   '_id': '6745d4c5ecd6d824cc57c98a',
   'slug': 'nike-running-shoes-sneakers-cw3398-102-womens-size-11-white-air-max-bella-tr-4-29cd1d62-c9cf-4e08-91f6-0bd041500200-1732629701027',
   'price': 41.99,
   'isVerified': False,
   'isBoosted': False,
   'source': 'shopify',
   'images': ['https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_8f6713b3-9859-4aff-9118-50541af25f47.jpg?v=1693248497',
    'https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_3c1bbfe7-1fe9-4df1-8d55-1ddc48d3bd8d.jpg?v=1693248497',
    'https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_ae1975b0-2b04-459e-8443-c178663631d7.jpg?v=1693248497',
    'https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_846093bc-814d-4a4f-869f-da942285943d.jpg?v=1693248497',
    'https://cdn.shopify.com/s/files/1/0077/1745/3881/p

In [ ]:
# Create a tool_result message using mk_funcres.
tool_result_msg = mk_funcres(tool_req.id, tool_result)
tool_result_msg  # This prints the tool_result message dictionary.

{'type': 'tool_result',
 'tool_use_id': 'toolu_01DUyj6gpXq6xE69T2exP8sp',
 'content': "{'status': 'success', 'Products': [{'name': 'Nike Running Shoes Sneakers CW3398-102 Womens Size 11 White Air Max Bella TR 4', 'storeName': 'Goodfair', 'brand': 'Nike', '_id': '6745d4c5ecd6d824cc57c98a', 'slug': 'nike-running-shoes-sneakers-cw3398-102-womens-size-11-white-air-max-bella-tr-4-29cd1d62-c9cf-4e08-91f6-0bd041500200-1732629701027', 'price': 41.99, 'isVerified': False, 'isBoosted': False, 'source': 'shopify', 'images': ['https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_8f6713b3-9859-4aff-9118-50541af25f47.jpg?v=1693248497', 'https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_3c1bbfe7-1fe9-4df1-8d55-1ddc48d3bd8d.jpg?v=1693248497', 'https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_ae1975b0-2b04-459e-8443-c178663631d7.jpg?v=1693248497', 'https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_846093bc-814d-4a4f-869f-da942285943d.jpg?v=1693248497', 'https:/

In [ ]:
# Call the chat without a new prompt so that the tool_result message is sent,
# and the conversation continues.
r2 = chat()
r2  # This should produce the updated assistant response incorporating the tool result.

Render product !!!!


Here are some Nike running shoes I found for you:

<details>

- id: `msg_01ECPd6FDW4ihbEsmuwT3Usm`
- content: `[{'citations': None, 'text': 'Here are some Nike running shoes I found for you:', 'type': 'text'}, {'id': 'toolu_01EznZMHPK4Y5Z638R3XsB1x', 'input': {'item': '{"name": "Nike Running Shoes Sneakers CW3398-102 Womens Size 11 White Air Max Bella TR 4", "storeName": "Goodfair", "brand": "Nike", "_id": "6745d4c5ecd6d824cc57c98a", "price": 41.99, "agoraScore": 92, "images": ["https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_8f6713b3-9859-4aff-9118-50541af25f47.jpg?v=1693248497", "https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_3c1bbfe7-1fe9-4df1-8d55-1ddc48d3bd8d.jpg?v=1693248497", "https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_ae1975b0-2b04-459e-8443-c178663631d7.jpg?v=1693248497", "https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_846093bc-814d-4a4f-869f-da942285943d.jpg?v=1693248497", "https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_7bec3532-a075-458e-bdf3-995fbc0ee317.jpg?v=1693248497", "https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_2f9d6a73-2dc7-4a41-95e7-88d2225940f8.jpg?v=1693248497", "https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_5d4a3443-86c5-431b-a14b-367681e5578c.jpg?v=1693248497", "https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_196146b6-a080-42bb-a5c7-cbef731fc79f.jpg?v=1693248497", "https://cdn.shopify.com/s/files/1/0077/1745/3881/products/57_1727d3c1-71d4-4d48-b225-e76ca5ce9940.jpg?v=1693248497"]}'}, 'name': 'render_product', 'type': 'tool_use'}]`
- model: `claude-3-7-sonnet-20250219`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 4610, 'output_tokens': 720}`

</details>

## Trials

In [ ]:
def ProductCard(
    item: str # JSON string containing product details (`name`, `price`, `brand`, `storeName`, `agoraScore`, `images`, `_id`)
):
    """
    Renders a product card UI component displaying product details clearly and attractively.
    Expects `item` as a JSON-formatted string with these keys:
    - `name`: Product name (str)
    - `price`: Product price (float or str)
    - `brand`: Brand name (str)
    - `storeName`: Store name (str)
    - `agoraScore`: Product rating score out of 100 (int or float)
    - `images`: List of image URLs (list[str]), first image used as main display
    - `_id`: Unique identifier for the product (str)
    """
    item = dict2obj(json.loads(item))
    return to_xml(
        Card(
            DivHStacked(
                # Left column with image
                Div(
                    Img(src=item.images[0], alt=item.name, cls="max-h-52 object-contain")
                ),
                # Right column with product details
                DivVStacked(
                    H3(item.name),
                    Strong(f"${item.price}"),
                    Div(f"Brand: {item.brand}"),
                    Div(f"Store: {item.storeName}"),
                    Div(f"Rating: {item.agoraScore}/100"),
                    Button("View Details", id=f"view-{item._id}")
                )
            )
        )
    )

In [ ]:
from fastcore.utils import *
from fasthtml.common import *
import fasthtml.common as fh
from functools import partial
import re
from ghapi.all import GhApi

#| hide
#| eval: false
from fasthtml.jupyter import *

# App Setup
if 'server' in globals(): server.stop()

#| export
custom_styling = (
    Style(':root { --gunmetal-dark: #1C2329; --text-light: #E0E0E0; }'),
    Style('body { padding-top: 80px; background: var(--gunmetal-dark); color: var(--text-light); }'),
    Style('''
        .floating-navbar {
            position: fixed;
            top: 20px;
            left: 50%;
            transform: translateX(-50%);
            width: 95%;
            max-width: 1200px;
            z-index: 1000;
        }
    '''),
    Style('''
        .floating-navbar-container:not(.uk-navbar-transparent) {
            background: var(--gunmetal-dark);
            border-radius: 15px;
            box-shadow: 0 4px 10px rgba(255,255,255,0.1);
            padding: 10px 0;
        }
    '''),
    Style('.uk-navbar-nav > li > a { color: var(--text-light); min-height: 50px; }'),
    Style('.uk-navbar-nav > li:hover > a { color: white; }'),
    Style('.active { text-decoration: underline !important; font-weight: bold; }'),
)

# Application Initialization
#| export
app, rt = fast_app(pico=False, hdrs=Theme.blue.headers())

server = JupyUvi(app, port=8001)
Show = partial(HTMX, app=app, link=True)

In [ ]:
data = search_trial('headphones')
for p in data['Products'][:4]:
    Show(ProductCard(json.dumps(p)))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()